In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import time
import requests
import random
import aiohttp

In [2]:
async def getJSONData(full_url):
    try:
        async with aiohttp.ClientSession() as session:
            async with session.get(full_url) as r:
                if r.status == 200:
                    js = await r.json()
                    return js
    except:
        return None




In [3]:
metadata = await getJSONData('https://www.mkwlounge.gg/api/ladderplayer.php?ladder_type=rt&all=1')
print(type(metadata))

<class 'dict'>


In [4]:
playerdata = pd.DataFrame(metadata['results'])
playerdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3951 entries, 0 to 3950
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   player_id                3951 non-null   int64  
 1   player_name              3951 non-null   object 
 2   player_country_flag      3951 non-null   object 
 3   discord_user_id          3404 non-null   object 
 4   base_mmr                 3951 non-null   int64  
 5   base_lr                  3951 non-null   int64  
 6   strikes                  3951 non-null   int64  
 7   game_counter             3951 non-null   int64  
 8   current_mmr              3951 non-null   int64  
 9   current_lr               3951 non-null   int64  
 10  peak_mmr                 3951 non-null   int64  
 11  peak_lr                  3951 non-null   int64  
 12  lowest_mmr               3951 non-null   int64  
 13  lowest_lr                3951 non-null   int64  
 14  wins                    

In [5]:
playerdata.head()

,player_id,player_name,player_country_flag,discord_user_id,base_mmr,base_lr,strikes,game_counter,current_mmr,current_lr,...,previous_ranking,previous_percentile,last_event_date,total_events_since_date,since_date,update_date,current_division,current_class,url,current_emblem
0,260,Az,jp,188508063995527168,17385,7574,2,14,17155,15757,...,1,99.9213,2022-04-27 14:37:46,54,None,2022-04-27 14:37:46,Grandmaster,Class X,https://www.mkwlounge.gg/ladder/player.php?player_id=260&ladder_id=3,https://i.imgur.com/ZV1JMyg.png
1,1207,Kevin,de,551040312148230144,14234,5817,3,12,14339,15268,...,2,99.8425,2022-04-28 17:17:25,127,None,2022-04-28 17:17:25,Grandmaster,Class X,https://www.mkwlounge.gg/ladder/player.php?player_id=1207&ladder_id=3,https://i.imgur.com/ZV1JMyg.png
2,290,barney,no,82852901382258688,15128,6403,0,16,14495,14979,...,3,99.7638,2022-04-28 19:27:32,94,None,2022-04-28 19:27:32,Master,Class X,https://www.mkwlounge.gg/ladder/player.php?player_id=290&ladder_id=3,https://i.imgur.com/BdAQPiE.png
3,1614,Kasperinos,us,195336607807700992,14615,6758,2,17,13902,14817,...,4,99.6850,2022-04-23 10:01:15,92,None,2022-04-23 10:01:15,Master,Class X,https://www.mkwlounge.gg/ladder/player.php?player_id=1614&ladder_id=3,https://i.imgur.com/BdAQPiE.png
4,1043,Fox,un,576472300006670374,15066,6480,1,14,13837,14535,...,5,99.6063,2022-04-27 13:39:15,92,None,2022-04-27 13:39:15,Master,Class X,https://www.mkwlounge.gg/ladder/player.php?player_id=1043&ladder_id=3,https://i.imgur.com/BdAQPiE.png


In [6]:
playerdata = playerdata[['player_id','player_name','current_mmr','current_lr','win_percentage','win10_percentage','average_score','average10_score','total_events']]
playerdata.head()

,player_id,player_name,current_mmr,current_lr,win_percentage,win10_percentage,average_score,average10_score,total_events
0,260,Az,17155,15757,0.962963,1.000000,99.0370,96.1000,54
1,1207,Kevin,14339,15268,0.769841,0.555556,89.2217,78.7500,127
2,290,barney,14495,14979,0.784946,0.600000,91.0257,90.7000,94
3,1614,Kasperinos,13902,14817,0.758242,0.400000,90.2771,76.9286,92
4,1043,Fox,13837,14535,0.684783,0.200000,92.1954,83.9000,92


In [7]:
def getTrackData(player_id):
    url = 'https://mkwlounge.gg/ladder/player.php?player_id={}&ladder_id=3'.format(player_id)
    # print(url)
    playerpage = requests.get(url, timeout=3)
    playersoup = bs(playerpage.content)
    playertables = playersoup.find_all('table')
    dfs = pd.read_html(str(playertables))[3]
    return dfs

In [8]:
def formatTrackData(df, index):
    df.drop(['Std Dev Time', 'Average Points', 'Fastest Time', 'Slowest Time'], axis=1, inplace=True)
    melted = df.pivot(index = 'Engine', columns='Track')
    melted = melted.drop('Mirror', axis=0, errors='ignore')
    melted = melted.drop('100cc', axis=0, errors='ignore')
    melted = melted.rename(index={'150cc': index})
    return melted

In [121]:
def renameCols(df):
    df.rename(columns=lambda x: upperDict[x], level=0, inplace=True)
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    df.drop(['AT -','AP -', 'R -'], axis=1, inplace=True)
    df.rename(columns=lambda c: c.replace(' (Nintendo)',''), inplace=True)
    df.index.name = None
    return df



In [10]:
betterdata = playerdata.loc[playerdata["total_events"]!= 0]
betterdata = betterdata.loc[betterdata["current_mmr"] >= 0]
betterdata.shape

(1254, 9)

In [40]:

test = pd.DataFrame()
dfs = []
i =0
for index, row in betterdata.iterrows():
    trackdata = getTrackData(row['player_id'])
    formatData = formatTrackData(trackdata, index)
    formatData['player_id'] = row['player_id']
    dfs.append(formatData)
    if i %100 == 0:
        print('working: ' + str(i))
    i = i + 1

working: 0
working: 100
working: 200
working: 300
working: 400
working: 500
working: 600
working: 700
working: 800
working: 900
working: 1000
working: 1100
working: 1200


In [122]:
test = pd.DataFrame()
test = pd.DataFrame().append(dfs)

In [123]:
test = renameCols(test)
test.head()

,AP DS Delfino Square,AP DS Desert Hills,AP DS Peach Gardens,AP DS Yoshi Falls,AP GBA Bowser Castle 3,AP GBA Shy Guy Beach,AP GCN DK Mountain,AP GCN Mario Circuit,AP GCN Peach Beach,AP GCN Waluigi Stadium,...,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine,player_id
0,5th,5th,5th,4th,5th,3rd,5th,4th,5th,5th,...,46.0,59.0,24.0,17.0,97.0,128.0,16.0,39.0,14.0,260
1,6th,5th,5th,5th,5th,5th,5th,6th,6th,6th,...,146.0,121.0,61.0,24.0,217.0,304.0,36.0,79.0,54.0,1207
2,6th,5th,6th,4th,5th,9th,5th,5th,6th,5th,...,61.0,67.0,22.0,21.0,87.0,122.0,18.0,42.0,34.0,290
3,5th,5th,5th,5th,5th,5th,5th,7th,5th,5th,...,55.0,43.0,26.0,19.0,102.0,104.0,7.0,36.0,22.0,1614
4,6th,5th,5th,6th,5th,3rd,4th,6th,6th,5th,...,58.0,54.0,20.0,15.0,62.0,104.0,13.0,31.0,22.0,1043


In [110]:
test.rename(columns=lambda x: upperDict[x], level=0, inplace=True)
test.columns.droplevel(1).unique()

Index(['AP', 'AT', 'R', 'player_id'], dtype='object')

In [124]:
betterdata.head()

,player_id,player_name,current_mmr,current_lr,win_percentage,win10_percentage,average_score,average10_score,total_events
0,260,Az,17155,15757,0.962963,1.000000,99.0370,96.1000,54
1,1207,Kevin,14339,15268,0.769841,0.555556,89.2217,78.7500,127
2,290,barney,14495,14979,0.784946,0.600000,91.0257,90.7000,94
3,1614,Kasperinos,13902,14817,0.758242,0.400000,90.2771,76.9286,92
4,1043,Fox,13837,14535,0.684783,0.200000,92.1954,83.9000,92


In [125]:
combodata = pd.merge(betterdata, test, left_on='player_id', right_on='player_id')
combodata.head()

,player_id,player_name,current_mmr,current_lr,win_percentage,win10_percentage,average_score,average10_score,total_events,AP DS Delfino Square,...,R Wii Koopa Cape,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine
0,260,Az,17155,15757,0.962963,1.000000,99.0370,96.1000,54,5th,...,61.0,46.0,59.0,24.0,17.0,97.0,128.0,16.0,39.0,14.0
1,1207,Kevin,14339,15268,0.769841,0.555556,89.2217,78.7500,127,6th,...,161.0,146.0,121.0,61.0,24.0,217.0,304.0,36.0,79.0,54.0
2,290,barney,14495,14979,0.784946,0.600000,91.0257,90.7000,94,6th,...,94.0,61.0,67.0,22.0,21.0,87.0,122.0,18.0,42.0,34.0
3,1614,Kasperinos,13902,14817,0.758242,0.400000,90.2771,76.9286,92,5th,...,73.0,55.0,43.0,26.0,19.0,102.0,104.0,7.0,36.0,22.0
4,1043,Fox,13837,14535,0.684783,0.200000,92.1954,83.9000,92,6th,...,70.0,58.0,54.0,20.0,15.0,62.0,104.0,13.0,31.0,22.0


In [45]:
combodata.loc[combodata['player_id'] == 4088]

,player_id,player_name,current_mmr,current_lr,win_percentage,win10_percentage,average_score,average10_score,total_events,"(Average Placement, -)",...,"(Races, Wii Koopa Cape (Nintendo))","(Races, Wii Luigi Circuit (Nintendo))","(Races, Wii Maple Treeway (Nintendo))","(Races, Wii Mario Circuit (Nintendo))","(Races, Wii Moo Moo Meadows (Nintendo))","(Races, Wii Moonview Highway (Nintendo))","(Races, Wii Mushroom Gorge (Nintendo))","(Races, Wii Rainbow Road (Nintendo))","(Races, Wii Toad's Factory (Nintendo))","(Races, Wii Wario's Gold Mine (Nintendo))"
422,4088,fordj262,4226,4116,0.75,0.75,85.0,85.0,4,NaN,...,1.0,1.0,1.0,NaN,1.0,NaN,2.0,1.0,2.0,NaN


In [46]:
combodata.shape

(1254, 108)

In [49]:
betterdata.shape

(1254, 9)

In [36]:
test.shape

(1254, 99)

In [48]:
combodata['player_id'].value_counts().max()

1

In [92]:
test.columns.droplevel(1)

Index(['Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Placement', 'Average Placement', 'Average Placement',
       'Average Time', 'Average Time', 'Average Time', 'Average Time',
       'Average Time', 'Average Time', 'Average Time', 'Average Time',
       'Average Time', 'Average Time', 'Average Time', 'Average Time',
       'Average T

In [126]:
t2 = pd.read_csv('datasets/loungedata')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/loungedata'

In [64]:
t2.columns = [' '.join(col).strip() for col in t2.columns.values]
t2.columns = [' '.replace('(nintendo)','') for col in t2.columns.values]

In [55]:
t2.drop(['Average Time -','Average Placement -', 'Races -'], axis=1, inplace=True)

In [65]:
t2.head()

,,,,,,,,,,,...,,,,,,,,,,
Engine,,,,,,,,,,,,,,,,,,,,,
0,5th,5th,5th,4th,5th,3rd,5th,4th,5th,5th,...,46.0,59.0,24.0,17.0,97.0,128.0,16.0,39.0,14.0,260
1,6th,5th,5th,5th,5th,5th,5th,6th,6th,6th,...,146.0,121.0,61.0,24.0,217.0,304.0,36.0,79.0,54.0,1207
2,6th,5th,6th,4th,5th,9th,5th,5th,6th,5th,...,61.0,67.0,22.0,21.0,87.0,122.0,18.0,42.0,34.0,290
3,5th,5th,5th,5th,5th,5th,5th,7th,5th,5th,...,55.0,43.0,26.0,19.0,102.0,104.0,7.0,36.0,22.0,1614
4,6th,5th,5th,6th,5th,3rd,4th,6th,6th,5th,...,58.0,54.0,20.0,15.0,62.0,104.0,13.0,31.0,22.0,1043
